In [1]:
import json
import numpy as np

In [90]:
def solve():
    global iterations, solutions
    for i, pos in enumerate(posisiton_list):
        if nation_list[i] == -1:
            (nation_set, league_set, club_sets, club_leagues) = get_possible_NLC(i)
            for nat in nation_set:
                for lea in league_set:
                    club_set = club_sets[club_leagues.index(lea)]
                    for clu in club_set:
                        nation_list[i] = nat
                        league_list[i] = lea
                        club_list[i] = clu
                        
                        calc_chemistry(i)
                        
                        count_list = [0] * len(posisiton_list)
                        n_count, count_list = nation_count(i, 0, count_list)
                        
                        count_list = [0] * len(posisiton_list)
                        l_count, count_list = league_count(i, 0, count_list)
                        
                        iterations += 1

                        if calc_constrain(n_count, l_count):                                
                            cont = solve()
                            if cont == 1:
                                return 1
            chemistry_list[i] = 1
            link_matrix[pos["connection_list"], i] = 3
            nation_list[i] = -1
            league_list[i] = -1
            club_list[i] = -1       
            return 0
    solutions += 1
    nations = count_nations()
    leagues = count_leagues()
    print("----", solutions, "----", iterations, "----")
#     print(link_matrix)
    print(disp_list)
    print(nation_list, "Nations")
    print(league_list, "Leagues")
    print(club_list, "Clubs")
    print(nations, leagues)
    
#     a = input("More (y)? ")
#     if a != "y":
#         return 1
    if solutions == 159:
        return 1
    return 0

In [48]:
def calc_constrain(n_count, l_count):
    if sum(chemistry_list) == len(chemistry_list):
        if n_count < 4:
            if l_count < 4:
                return True
    return False

In [62]:
def nation_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] == nation_list[idx] and count_list[conn] == 0:
            count, count_list = nation_count(conn, count, count_list)
    return count, count_list

def league_count(idx, count, count_list):
    count += 1
    count_list[idx] = 1
    for conn in posisiton_list[idx]["connection_list"]:
        if league_list[conn] == league_list[idx] and count_list[conn] == 0:
            count, count_list = league_count(conn, count, count_list)
    return count, count_list

In [93]:
def count_nations():
    n_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            n_c += 1
            (c, count_list) = nation_count(i, 0, count_list)
    return n_c

def count_leagues():
    l_c = 0
    count_list = [0] * len(posisiton_list)
    for i, pos in enumerate(posisiton_list):
        if count_list[i] == 0:
            l_c += 1
            (c, count_list) = league_count(i, 0, count_list)
    return l_c

In [5]:
def calc_chemistry(idx):
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            link = 0
            if nation_list[conn] == nation_list[idx]:
                link += 1
            if league_list[conn] == league_list[idx]:
                link += 1
                if club_list[conn] == club_list[idx]:
                    link += 1
            link_matrix[idx, conn] = link
            link_matrix[conn, idx] = link
            chemistry_list[conn] = min(1, sum(link_matrix[conn, :]) / len(posisiton_list[conn]["connection_list"]))
    chemistry_list[idx] = min(1, sum(link_matrix[idx, :]) / len(posisiton_list[idx]["connection_list"]))

In [6]:
def get_possible_NLC(idx):
    nation_set = set()
    league_set = set()
    club_leagues = []
    club_sets = []
    for conn in posisiton_list[idx]["connection_list"]:
        if nation_list[conn] != -1:
            nation_set.add(nation_list[conn])
            league_set.add(league_list[conn])
            if league_list[conn] in club_leagues:
                club_sets[club_leagues.index(league_list[conn])].add(club_list[conn])
            else:
                club_leagues.append(league_list[conn])
                club_sets.append({club_list[conn]})
    
    for i, s in enumerate(club_sets):
        club_sets[i] = s
        club_sets[i].add(max(club_sets[i]) + 1)
        
    if not nation_set:
        nation_set.add(0)
    else:
        nation_set.add(max(nation_set) + 1)

    if not league_set:
        league_set.add(0)
        club_leagues.append(0)
        club_sets.append({0})
    else:
        league_set.add(max(league_set) + 1)
        club_leagues.append(max(league_set))
        club_sets.append({0})
    return (nation_set, league_set, club_sets, club_leagues)

In [91]:
formation_index = 4

f = open("../files/formations.json")
d = json.load(f)
posisiton_list = d["formations"][formation_index]["positions_list"]
print(d["formations"][formation_index]['name'])
n_pos = len(posisiton_list)
disp_list = []
for i in posisiton_list:
    if 'side' in i:
        disp_list.append(i['side'] + i['position'])
    else:
        disp_list.append(i['position'])

1-2-1-1


In [94]:
chemistry_list = [1] * len(posisiton_list)
nation_list = [-1] * len(posisiton_list)
league_list = [-1] * len(posisiton_list)
club_list = [-1] * len(posisiton_list)
link_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(posisiton_list):
    link_matrix[pos["connection_list"], i] = 3
    
iterations = 0
solutions = 0
solve()

---- 1 ---- 10 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 0, 1, 1] Leagues
[0, 0, 0, 0, 0] Clubs
2 2
---- 2 ---- 25 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 0, 1, 1] Leagues
[0, 0, 1, 0, 0] Clubs
2 2
---- 3 ---- 40 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 1, 1, 1] Leagues
[0, 0, 0, 0, 0] Clubs
2 2
---- 4 ---- 41 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 1, 1, 1] Leagues
[0, 0, 0, 0, 1] Clubs
2 2
---- 5 ---- 42 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 1, 1, 2] Leagues
[0, 0, 0, 0, 0] Clubs
2 3
---- 6 ---- 43 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 2] Nations
[0, 0, 1, 1, 1] Leagues
[0, 0, 0, 0, 0] Clubs
3 2
---- 7 ---- 44 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 2] Nations
[0, 0, 1, 1, 1] Leagues
[0, 0, 0, 0, 1] Clubs
3 2
---- 8 ---- 47 ----
['RM', 'CB', 'LM', 'CAM', 'ST']
[0, 0, 0, 1, 1] Nations
[0, 0, 1, 1, 1] Leagues
[0, 0, 0, 1, 1] Cl

1

Add results to pandas<br>
one row for each result<br>
three coloumns for each position (N, L, C) (33 in total)<br>
count unique N, L and C, and add these as coloumns<br>
- how to count clubs?